In [2]:
#importing openslide:
OPENSLIDE_PATH = r"C:\Users\Kevin\Downloads\openslide-win64-20221217\bin"

import os
if hasattr(os, 'add_dll_directory'):
    # Python >= 3.8 on Windows
    with os.add_dll_directory(OPENSLIDE_PATH):
        import openslide
else:
    import openslide

In [2]:
import numpy as np
from PIL import Image
import math
import os
import pandas as pd
from tqdm import tqdm

In [3]:
# First get list of all deparafinized and student_score >2 and CLUE:
all_df_src = r'\\shelter\Kyu\unstain2stain\unstain2stain_wsi\wsi_list_230314.xlsx'
all_df = pd.read_excel(all_df_src)
desired_df_name = all_df[all_df.scan_batch == 3] # CLUE
desired_df_name1 = all_df[all_df.scan_batch == 4] # CLUE
desired_name = list(desired_df_name.wsi_name)
desired_name1 = list(desired_df_name1.wsi_name)
desired_names = desired_name + desired_name1
he_desired_names = [x for x in desired_names if x[-3:] == "_he"]
he_desired_names_src = [x+".ndpi" for x in he_desired_names]
us_desired_names = [x for x in desired_names if not x[-3:] == "_he"]
us_desired_names_src = [x+".ndpi" for x in us_desired_names]
US_dst_src = r'\\shelter\Kyu\unstain2stain\tiles\unregistered_tiles\US'
US_ndpi_src = r'\\shelter\Kyu\unstain2stain\unstain2stain_wsi\Unstained'
US_desired_src = [os.path.join(US_ndpi_src,x) for x in us_desired_names_src]
US_desired_src

In [ ]:
for idx, src in tqdm(enumerate(US_desired_src),total=len(US_desired_src),colour='red',desc='WSI Processed'):
    wsi_name = us_desired_names[idx]
    save_src = os.path.join(US_dst_src,wsi_name)
    if not os.path.exists(save_src):
        os.makedirs(save_src)
    slide = openslide.open_slide(src)
    target_level = 0
    target_dim = slide.level_dimensions[target_level]
    width, height = target_dim
    # unpad since it's white background anyways
    tiles_x_unpad = int((width % 1024) / 2)
    tiles_y_unpad = int((height % 1024) / 2)
    # read from 0+tiles_x_unpad/y_pad and max-tiles_x_unpad/ypad:
    num_rows= ((width - tiles_x_unpad) - (0 + tiles_x_unpad)) / 1024
    num_cols= ((height - tiles_y_unpad) - (0 + tiles_y_unpad)) / 1024
    num_rows = int(num_rows)
    num_cols = int(num_cols)
    print(num_rows)
    print(num_cols)
        # tile to 1024 x 1024
    for row_idx in range(num_rows): # 0~num_rows-1
        for col_idx in range(num_cols): # 0~num_cols-1
            xcoord= tiles_x_unpad + 1024 * row_idx
            ycoord= tiles_y_unpad + 1024 * col_idx
            nsimage = slide.read_region(location=(xcoord,ycoord),level=target_level,size=(1024,1024))
            nsimage = np.array(nsimage)
            imgidx = row_idx * num_cols + (col_idx + 1)
            imgidx = f"{imgidx:04d}"
            imgcoord = str(xcoord) + "_" + str(ycoord) + "xy"
            imgname = imgcoord + imgidx + ".png"
            tile_save_src = os.path.join(save_src,imgname)
            Image.fromarray(nsimage).save(tile_save_src)

### Parallel processing:

In [ ]:
import numpy as np
from PIL import Image
import math
import os
import pandas as pd
from tqdm import tqdm
import pathos
from pathos.multiprocessing import ProcessPool

In [ ]:
pathos.helpers.cpu_count()

In [10]:
# First get list of all deparafinized and student_score >2 and CLUE:
all_df_src = r'\\shelter\Kyu\unstain2stain\unstain2stain_wsi\wsi_list_230314.xlsx'
all_df = pd.read_excel(all_df_src)
desired_df_name = all_df[all_df.scan_batch == 3] # CLUE
desired_df_name1 = all_df[all_df.scan_batch == 4] # CLUE
desired_name = list(desired_df_name.wsi_name)
desired_name1 = list(desired_df_name1.wsi_name)
desired_names = desired_name + desired_name1
he_desired_names = [x for x in desired_names if x[-3:] == "_he"]
he_desired_names_src = [x+".ndpi" for x in he_desired_names]
us_desired_names = [x for x in desired_names if not x[-3:] == "_he"]
us_desired_names_src = [x+".ndpi" for x in us_desired_names]
US_dst_src = r'\\shelter\Kyu\unstain2stain\tiles\unregistered_tiles\US'
US_ndpi_src = r'\\shelter\Kyu\unstain2stain\unstain2stain_wsi\Unstained'
US_desired_src = [os.path.join(US_ndpi_src,x) for x in us_desired_names_src]
US_desired_src

['\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\Unstained\\OTS_14684_1.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\Unstained\\OTS_14684_2.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\Unstained\\OTS_14684_3.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\Unstained\\OTS_14684_4.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\Unstained\\OTS_14684_5.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\Unstained\\OTS_14684_6.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\Unstained\\OTS_14684_7.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\Unstained\\OTS_14684_8.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\Unstained\\OTS_14684_9.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\Unstained\\OTS_14832_1.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\Unstained\\OTS_14832_2.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\Unstained\\OTS_14832_3

In [4]:
#importing openslide:
OPENSLIDE_PATH = r"C:\Users\labuser1\Downloads\openslide-win64-20230414\bin"

import os
if hasattr(os, 'add_dll_directory'):
    # Python >= 3.8 on Windows
    with os.add_dll_directory(OPENSLIDE_PATH):
        import openslide
else:
    import openslide

In [20]:
def tile_wsi(src, wsi_name, save_src):
    import os
    import numpy as np
    from PIL import Image
    import math
    import pandas as pd
    from tqdm import tqdm
    from pathos.multiprocessing import ProcessPool
    OPENSLIDE_PATH = r"C:\Users\labuser1\Downloads\openslide-win64-20230414\bin"
    if hasattr(os, 'add_dll_directory'):
        # Python >= 3.8 on Windows
        with os.add_dll_directory(OPENSLIDE_PATH):
            import openslide
    else:
        import openslide
    if not os.path.exists(save_src):
        os.makedirs(save_src)
    slide = openslide.open_slide(src)
    target_level = 0
    target_dim = slide.level_dimensions[target_level]
    width, height = target_dim
    # unpad since it's white background anyways
    tiles_x_unpad = int((width % 1024) / 2)
    tiles_y_unpad = int((height % 1024) / 2)
    # read from 0+tiles_x_unpad/y_pad and max-tiles_x_unpad/ypad:
    num_rows= ((width - tiles_x_unpad) - (0 + tiles_x_unpad)) / 1024
    num_cols= ((height - tiles_y_unpad) - (0 + tiles_y_unpad)) / 1024
    num_rows = int(num_rows)
    num_cols = int(num_cols)
    # tile to 1024 x 1024
    for row_idx in range(num_rows): # 0~num_rows-1
        for col_idx in range(num_cols): # 0~num_cols-1
            xcoord= tiles_x_unpad + 1024 * row_idx
            ycoord= tiles_y_unpad + 1024 * col_idx
            nsimage = slide.read_region(location=(xcoord,ycoord),level=target_level,size=(1024,1024))
            nsimage = np.array(nsimage)
            imgidx = row_idx * num_cols + (col_idx + 1)
            imgidx = f"{imgidx:04d}"
            imgcoord = str(xcoord) + "_" + str(ycoord) + "xy"
            imgname = imgcoord + imgidx + ".png"
            tile_save_src = os.path.join(save_src, imgname)
            Image.fromarray(nsimage).save(tile_save_src)

In [ ]:
pool = ProcessPool(nodes=8)
results = []

for idx, src in tqdm(enumerate(US_desired_src), total=len(US_desired_src), colour='red', desc='WSI Processed'):
    wsi_name = us_desired_names[idx]
    save_src = os.path.join(US_dst_src, wsi_name)
    results.append(pool.apipe(tile_wsi, src, wsi_name, save_src))

# Get the results
for result in results:
    import os
    result.get()

# Close the process pool
pool.close()

### BELOW IS WORKFLOW TO CREATE UNSTAINED TILES MUCH FASTER, BUT ONLY WORKS ON >128GB RAM. MUCH FASTER THAN THE METHOD ABOVE WHICH TAKES >6 HRS TO GENERATE TILE FOR ONE IMAGE!

### Setup:

In [7]:
#importing openslide:
OPENSLIDE_PATH = r"C:\Users\labuser1\Downloads\openslide-win64-20230414\bin"

import os
if hasattr(os, 'add_dll_directory'):
    # Python >= 3.8 on Windows
    with os.add_dll_directory(OPENSLIDE_PATH):
        import openslide
else:
    import openslide

In [ ]:
import numpy as np
from PIL import Image
import math
import os
import pandas as pd
from tqdm import tqdm
import time

In [3]:
def de_pad_ndpi(ndpi_src, tile_size):
    slide = openslide.open_slide(ndpi_src)
    dim = slide.dimensions
    width = dim[0]
    height = dim[1]
    de_pad_width = int(round((width % tile_size)/2))
    de_pad_height = int(round((height % tile_size)/2))
    de_pad_img = slide.read_region(location = (de_pad_width,de_pad_height),level=0, size=(width-2*de_pad_width,height-2*de_pad_height))
    return de_pad_img

In [4]:
def reshape_split(ndpi_src,tile_size):
    """

    Args:
        image: nparray of image
        kernel_size: desired tile size, 1024 x 1024 for example
    Requires the nparray height and width to be divisible by kernel_size tuple to be reshaped. De-pad to remove some white background (assuming ndpi has white background) if not divisible so that nparray becomes divisible using above function.
    Returns:
    Array of tiles that can be saved all at once.
    """
    slide = openslide.open_slide(ndpi_src)
    dim = slide.dimensions
    width = dim[0]
    height = dim[1]
    de_pad_width = int(round((width % tile_size)/2))
    de_pad_height = int(round((height % tile_size)/2))
    print("Using Openslide to read in image")
    if not(de_pad_width==0 and de_pad_height ==0): # if both width and height not divisible
        image = de_pad_ndpi(ndpi_src,tile_size)
    else:
        image = slide.read_region(location=(0,0),level=0,size=dim)
    image = np.array(image)
    image = Image.fromarray(image).convert(mode='RGB')
    image = np.array(image)
    img_height,img_width,channels=image.shape
    tiled_array = image.reshape(int(img_height/tile_size),
                                int(img_width/tile_size), tile_size,
                                tile_size,
                                channels)
    print("Tiled array shape is {}:".format(tiled_array.shape))
    return tiled_array, (de_pad_width,de_pad_height)

### Run the workflow:

In [5]:
# First get list of all deparafinized and student_score >2 and CLUE:
all_df_src = r'\\shelter\Kyu\unstain2stain\unstain2stain_wsi\wsi_list_230314.xlsx'
all_df = pd.read_excel(all_df_src)
desired_df_name = all_df[all_df.scan_batch == 3] # CLUE
desired_df_name1 = all_df[all_df.scan_batch == 4] # CLUE
desired_name = list(desired_df_name.wsi_name)
desired_name1 = list(desired_df_name1.wsi_name)
desired_names = desired_name + desired_name1
he_desired_names = [x for x in desired_names if x[-3:] == "_he"]
he_desired_names_src = [x+".ndpi" for x in he_desired_names]
us_desired_names = [x for x in desired_names if not x[-3:] == "_he"]
us_desired_names_src = [x+".ndpi" for x in us_desired_names]

In [ ]:
### Tile unstained images:
US_dst_src = r'\\shelter\Kyu\unstain2stain\tiles\unregistered_tiles\US'
US_ndpi_src = r'\\shelter\Kyu\unstain2stain\unstain2stain_wsi\Unstained'
US_desired_src = [os.path.join(US_ndpi_src,x) for x in us_desired_names_src]
for idx, src in tqdm(enumerate(US_desired_src),total=len(US_desired_src),colour='red',desc='WSI Processed'):
    wsi_name = us_desired_names[idx]
    save_src = os.path.join(US_dst_src,wsi_name)
    if not os.path.exists(save_src):
        os.makedirs(save_src)
    tiled_array, (de_pad_width,de_pad_height) = reshape_split(src,1024)
    start = time.time()
    print("Saving tiles from 5d Array")
    for row_idx in range(tiled_array.shape[1]):
        for col_idx in range(tiled_array.shape[0]):
            x_coord = row_idx*1024 + de_pad_width
            y_coord = col_idx*1024 + de_pad_height
            label= row_idx * tiled_array.shape[0] + (col_idx + 1)
            label = f"{label:04d}"
            tile_save_src = os.path.join(save_src,f'{x_coord}_{y_coord}xy{label}.png')
            tile = tiled_array[col_idx][row_idx]
            Image.fromarray(tile).save(tile_save_src)
    end = time.time()
    print("time it took to save all tiles: {} minutes".format((end-start)/60))

In [ ]:
### CHANGE ABOVE TO MULTIPROCESSING, AND SAVE THE 5D ARRAY TO FEATHER (5D ARRAY -> DF -> FEATHER)

In [11]:
start = time.time()
US_dst_src = r'\\shelter\Kyu\unstain2stain\tiles\unregistered_tiles\US'
US_ndpi_src = r'\\shelter\Kyu\unstain2stain\unstain2stain_wsi\Unstained'
wsi_name = us_desired_names[0]
US_desired_src = os.path.join(US_ndpi_src,wsi_name + ".ndpi")
save_src = os.path.join(US_dst_src,wsi_name)
if not os.path.exists(save_src):
    os.makedirs(save_src)
tiled_array, (de_pad_width,de_pad_height) = reshape_split(US_desired_src,1024)
print(tiled_array.shape)
end = time.time()
print("time it took to save all tiles: {} minutes".format((end-start)/60))

Using Openslide to read in image
Tiled array shape is (96, 161, 1024, 1024, 3):
(96, 161, 1024, 1024, 3)
time it took to save all tiles: 12.830497594674428 minutes


In [12]:
start = time.time()
for row_idx in range(tiled_array.shape[1]):
    for col_idx in range(tiled_array.shape[0]):
        x_coord = row_idx*1024 + de_pad_width
        y_coord = col_idx*1024 + de_pad_height
        label= row_idx * tiled_array.shape[0] + (col_idx + 1)
        label = f"{label:04d}"
        tile_save_src = os.path.join(save_src,f'{x_coord}_{y_coord}xy{label}.png')
        tile = tiled_array[col_idx][row_idx]
        Image.fromarray(tile).save(tile_save_src)
end = time.time()
print("time it took to save all tiles: {} minutes".format((end-start)/60))

KeyboardInterrupt: 

In [ ]:
### Tile HE images:
HE_dst_src = r'\\shelter\Kyu\unstain2stain\tiles\unregistered_tiles\HE'
HE_ndpi_src = r'\\shelter\Kyu\unstain2stain\unstain2stain_wsi\HE'
HE_desired_src = [os.path.join(HE_ndpi_src,x) for x in he_desired_names_src]
for idx, src in tqdm(enumerate(HE_desired_src),total=len(HE_desired_src),colour='red',desc='WSI Processed'):
    wsi_name = he_desired_names[idx]
    save_src = os.path.join(HE_dst_src,wsi_name)
    if not os.path.exists(save_src):
        os.makedirs(save_src)
    tiled_array, (de_pad_width,de_pad_height) = reshape_split(src,1024)
    start = time.time()
    for row_idx in range(tiled_array.shape[1]):
        for col_idx in range(tiled_array.shape[0]):
            print("Saving tiles from 5d Array")
            x_coord = row_idx*1024 + de_pad_width
            y_coord = col_idx*1024 + de_pad_height
            label= row_idx * tiled_array.shape[0] + (col_idx + 1)
            label = f"{label:04d}"
            tile_save_src = os.path.join(save_src,f'{x_coord}_{y_coord}xy{label}.png')
            tile = tiled_array[col_idx][row_idx]
            Image.fromarray(tile).save(tile_save_src)
    end = time.time()
    print("time it took to save all tiles: {} minutes".format((end-start)/60))

### Now run inference on these tiles and then stitch the resulting tiles using stitch notebook
